In [ ]:
#!/usr/bin/env python3
import numpy as np 
import IO.user_provided
import objective.setup_objective
import optimizer.gradient_free

In [17]:
!cat "run_NelderMead"

#!/bin/bash
#SBATCH -J ML-BOP
#SBATCH -o out.o%j  
#SBATCH -e out.e%j 
#SBATCH -t 5-2:57:33
#SBATCH -n 12 -N 1   




# --------------------------------- Command Line Options --------------------------------

# -c (cores) "number of cores assigned"
# -j (jobid) "slurm jobid"
# -m (mode)  "debug" or "run" ( Optional ) 
# -i (input) "input file" 
# -Ref reference data address ( Optional )  
# -prep prepsystem or template folder ( Optional )   
# $SLUMR_NTASKS: Number of cores requested from SBATCH -n 
# $SLUMR_JOBID: JOB ID assigned after submitting the job 

./optimize -c $SLURM_NTASKS -j test_$SLURM_JOBID -i in_obj -Ref ../ReferenceData -prep ../prepsystem  


In [ ]:
# Taking input from command line or interactively

#logger, TOTAL_CORES,INPUT,JOBID = IO.user_provided.from_command_line(jobID="1234",total_cores=12,input_file="in_obj").finish_reading() 
logger, TOTAL_CORES,INPUT,JOBID = IO.user_provided.from_command_line().finish_reading() 

In [ ]:
# initialize properties matching  

ref_dict,predict_dict,argument_dict,LAMMPS,last_line = objective.setup_objective.setup(INPUT,TOTAL_CORES,JOBID,overwrite=True).finish()  


""" ref_dict is a python dictionary 

    { 
        matching_type: { sub_folder: [ address of ReferenceData ... ]}
        matching_type: { sub_folder: [ address of ReferenceData ... ]}
        .....
       
       } 
"""  

""" ref_dict is a python dictionary 

    { 
        matching_type: { sub_folder: [ address of predicted data ... ]}
        matching_type: { sub_folder: [ address of predicted data ... ]}
        .....
       
       } 
"""  

""" argument_dict is a python dictionary 

    { 
    
        matching_type: { sub_folder: (weight, cores for running sampling, cores for doing analysis, argument) } 
        matching_type: { sub_folder: (weight, cores for running sampling, cores for doing analysis, argument) } 
        ....
        
        } 

"""  

In [ ]:
# How does sampling method work. 

force_field = np.array([1.0,1.0,0.0,77638.534354,16.148387, -0.471029, 0.770018, 1e-06, 2.19964, 10921.97125, 3.282761, 0.270511, 2.750522,38840.92139 ]) 

LAMMPS.run("tersoff",force_field ) 

job_successfull = LAMMPS.exit() 


In [18]:
!cat "sample.log"

----------------------------- Initialize isobar matching -----------------------------


The sub_folder name: mW

The weight of objective function : 2.000 

Number of cores for running sampling: 12 

Number of cores for computing objective: 1

The other arbitrary argument: penalty 5 

------------------------- Initialize sampling method: LAMMPS -------------------------


Number of jobs: 6 

Number of cores used per job:  2 

Command:  module load intel && srun -n 2 --mpi=pmi2 --nodes=1 lmp_ml_water < in.isobar 

ERROR: Command: module load intel && srun -n %d --mpi=pmi2 --nodes=1 lmp_ml_water < in.%s, Folders: /project/palmer/Jingxiang/ours_optimization/tutorial/Tutorial-2---objective/run/test_366658/Predicted/isobar/mW/230 


In [ ]:
# create objective function that will be passed to optimizer

"""
matching_objective = objective.compute_objective(ref_dict,predict_dict,argument_dict,LAMMPS) 
"""

# objective function has the method of "optimize" 

# matching_objective.optimize("force-field parameters",force-field parameters) 

In [ ]:
# Run optimization using the objective:

optimize_Himmelblau = optimizer.gradient_free.NelderMeadSimplex(INPUT,matching_objective,skipped=last_line)

optimize_Himmelblau.run_optimization()